In [1]:
import numpy as np
import copy  as cpy
import queue

new_inst_tst=1
# maps to remove
tst_local_search=0
greedy_tst=0
updated_iter=1000
feasable_zone=1
infeasable_zone=0
# Unused flags
best_improvement=1
first_improvement=0
test=0
test_100=0
test_150=0
test_200=0

In [2]:
def read_txt2(fileLocation):
    with open(fileLocation,'r') as file:
        dimSubset,dimGroundSet,maxBudget = [int(i) for i in file.readline().split()] 
        # weights
        subsets_weights=np.array( [int(i) for i in file.readline().split()])
        #print(dimSubset,dimGroundSet,maxBudget)
        # profits
        ground_set_profits=np.array( [int(i) for i in file.readline().split()])
        objPerSubset={}
        for sub in range(dimSubset):
            objPerSubset[sub]=[]
        relation_matrix=np.array( [int(i) for i in file.readline().split()])
        for sub in range(dimSubset):
            for obj in range(sub*dimGroundSet,(sub+1)*dimGroundSet):
                if relation_matrix[obj]==1:
                    objPerSubset[sub].append(obj-dimGroundSet*sub)
        return objPerSubset,subsets_weights,ground_set_profits,maxBudget

#(objPerSubset,subsets_weights,ground_set_profits,maxBudget)=read_txt2("./instances2/bmcp_1100_1000_0.3_3000.txt")
#print(objPerSubset)

In [3]:
# calculate the score of a subset
def calcSubsetScore(subsetID,subsetWeight,subsetObjs,subsetPerObj,objScores):
    # Score= Sum of Value of Objects ( not yet added) /weight of the subset
    score=0
    for obj in subsetObjs: 
        if len(subsetPerObj[obj])==0:
            score+=objScores[obj]
    return score/subsetWeight[subsetID]


In [4]:
# until we have budget ( remaining_budget> max(not selected subsets)) 
#  Find the best subset 
#  Update the score the solution and budget
def greedy(subsets,subsetWeights,objScores, budget):
    # generate the list of all the subsets
    subset_ids=list(range(len(subsets)))
    #print(type(subset_ids[0]))
    # last element == the most expensive 
    subset_ids.sort(key=lambda x:subsetWeights[x])
    
    #print("ordered for wrights")
    #for sub in subset_ids:
    #    print("id",sub,"w",subsetWeights[sub])
    
    # init the map of sunsets per objects
    subsetPerObj={}
    remaining_budget=budget
    for obj in range(len(objScores)):
        subsetPerObj[obj]=[]
    # init the score
    score=0
    #init the solution( the list of selected subs)
    solution=np.array([])
    # helper vars
    (best_score,best_sub,curr_score)=(-1,0,0)
    while 1:
        best_score,curr_score=0,0
        for sub in subset_ids:
            curr_score=calcSubsetScore(sub,subsetWeights,subsets[sub],subsetPerObj,objScores)
            # if the score is better
            if curr_score>best_score:
                # update the score
                best_score=curr_score
                # save the id
                best_sub=sub
        if best_score==0:
            #print("No need to update")
            break
        #print("Adding ",best_sub)
        # update the score and the subs per book
        for obj in subsets[best_sub]:
            if len(subsetPerObj[obj])==0:
                score+=objScores[obj]
            subsetPerObj[obj].append(best_sub)
        #insert the element in the solution
        solution=np.append(solution,best_sub)
        # remove the element
        #print("removing",best_sub)
        subset_ids.remove(best_sub)
        #print("Remaining subs" ,subset_ids)
        # update the budg
        budget-=subsetWeights[best_sub]
        # stop if we've ended the subsets or the budget
        if len(subset_ids)==0:
            #print("Ended")
            break
        if budget<subsetWeights[subset_ids[len(subset_ids)-1]]:
            #print(" Remainig budget", budget," with cap", subsetWeights[subset_ids[len(subset_ids)-1]])
            break
            
    return (score,subsetPerObj,solution.astype(int))
    

In [5]:
def localSearchUnoptimized(score,solution,subsetPerObj,objPerSubset,subsetWeights,objScores,budget,zone,tabu_list):
    # The current budget and the one used in cycles
    restorer_budget,current_budget=sum(subsetWeights[sub] for sub in solution),0
    # Variables used for doing the swap and update the budget
    to_insert_cost,to_insert_id,to_rmv_id,to_rmv_cost=0,0,0,0
    # Score of the solution(with the penality factor), score without the penality factor
    new_score,real_score=0,0
    # Values used for returning
    to_return_cost,to_return_score,to_return_real_score,toReturnSPO,swapped=0,-1,0,{},(0,0)
    # FLag used for checking if there was at least one swap
    found=0
    
    # for each subset not in list or in the tabu list
    for sub in range(len(subsetWeights)):
        if not sub in solution and not sub in list(tabu_list.queue):
            
            # id and weight of the attemp
            to_insert_id,to_insert_cost=sub,subsetWeights[sub]
            # try to swap it with another subset
            for k in range(len(solution)):
                to_rmv_id,to_rmv_cost=solution[k],subsetWeights[solution[k]]
                copiedSPO=cpy.deepcopy(subsetPerObj)
                # try to swap it calculating the new weight
                current_budget=restorer_budget-to_rmv_cost+to_insert_cost
                #if zone==infeasable_zone or (zone==feasable_zone and budget>=new_cost):

                # always to init with the score
                real_score=score
               # print(" BU score",real_score)
                # for updating the score we need :
                
                # 1-remove all the obj from the leaving subset 
                for obj in objPerSubset[to_rmv_id]:
                    copiedSPO[obj].remove(to_rmv_id)
                    if len(copiedSPO[obj])==0:
                        real_score-=objScores[obj]

                # 2- Add the new objects
                for obj in objPerSubset[to_insert_id]:
                    if len(copiedSPO[obj])==0:
                        real_score+=objScores[obj]
                    copiedSPO[obj].append(to_insert_id)
                        
                # sub the penality factor
                new_score=real_score-(int(current_budget/budget)*max(0,current_budget-budget))
                if new_score>to_return_score:
                    # REMEMBER : SWAPPED[0] IS THE IDX IN SOLUTION ARRAY, NOT THE ID
                    to_return_real_score=real_score
                    swapped=(k,sub)
                    to_return_score=new_score
                    found=1
                    to_return_cost=current_budget
                    toReturnSPO=cpy.deepcopy(copiedSPO)
    if found:
        # now make the actual swap
        to_rmv_id=solution[swapped[0]]
        to_insert_id=swapped[1]
        solution[swapped[0]]=to_insert_id

        #CHECKING VALIDITY
        #obj_invalids=[]
        #tot_score=0
        #for to_check in solution:
        #    for obj in objPerSubset[to_check]:
        #        if not obj in obj_invalids:
        #            obj_invalids.append(obj)
        #            tot_score+=objScores[obj]
        #if tot_score==to_return_real_score:
        #    print("VALID SWAP")
            #print(to)
        #else:
        #    print("INVALID SWAPT",tot_score,to_return_real_score)

        # update the tabu list 
        if tabu_list.full():
            tabu_list.get()
            tabu_list.put(to_rmv_id)
        else:
            tabu_list.put(to_rmv_id)
        # and finally end
    return int(to_return_score),to_return_real_score,toReturnSPO,solution,to_return_cost,found
                        

In [6]:
def localSearch(score,solution,subsetPerObj,objPerSubset,subsetWeights,objScores,budget,zone,tabu_list):
    # The current budget and the one used in cycles
    restorer_budget,current_budget=sum(subsetWeights[sub] for sub in solution),0
    # Variables used for doing the swap and update the budget
    to_insert_cost,to_insert_id,to_rmv_id,to_rmv_cost=0,0,0,0
    # Score of the solution(with the penality factor), score without the penality factor
    new_score,real_score=0,0
    # Values used for returning
    to_return_cost,to_return_score,to_return_real_score,swapped=0,-1,0,(0,0)
    # FLag used for checking if there was at least one swap
    found=0
    
    # for each subset not in list or in the tabu list
    for sub in range(len(subsetWeights)):
        if not sub in solution and not sub in list(tabu_list.queue):
            
            # id and weight of the attemp
            to_insert_id,to_insert_cost=sub,subsetWeights[sub]
            # try to swap it with another subset
            for k in range(len(solution)):
                to_rmv_id,to_rmv_cost=solution[k],subsetWeights[solution[k]]
                # try to swap it calculating the new weight
                current_budget=restorer_budget-to_rmv_cost+to_insert_cost
                # Now try to swap if we're looking in the inf neighborhood or in the the feas neighborhood,
                # in the last case the feas condition budget>=current_budget must be respected
                if zone==infeasable_zone or (zone==feasable_zone and budget>=current_budget):
                    # always to init with the score
                    real_score=score
                    # print(" BU score",real_score)
                    # for updating the score we need :

                    # 1-remove all the obj from the leaving subset
                    # I won't remove it but only check if the removed sub was the only one with the obj
                    # and in case update the score
                    for obj in objPerSubset[to_rmv_id]:
                        # in case it was the only subset with the object and will not be inserted again
                        if len(subsetPerObj[obj])==1 and not obj in objPerSubset[to_insert_id]:
                            real_score-=objScores[obj]

                    # 2- To add the score of the new objects added from the inserted subs
                    for obj in objPerSubset[to_insert_id]:
                        # in case there is no subset 
                        if len(subsetPerObj[obj])==0:
                            real_score+=objScores[obj]

                    # sub the penality factor
                    new_score=real_score-(int(current_budget/budget)*max(0,current_budget-budget))
                    if new_score>to_return_score:
                        # REMEMBER : SWAPPED[0] IS THE IDX IN SOLUTION ARRAY, NOT THE ID
                        to_return_real_score=real_score
                        swapped=(k,sub)
                        to_return_score=new_score
                        found=1
                        to_return_cost=current_budget
    if found:
        # now make the actual swap
        to_rmv_id=solution[swapped[0]]
        to_insert_id=swapped[1]

        # 1-remove all the obj from the leaving subset 
        for obj in objPerSubset[to_rmv_id]:
            subsetPerObj[obj].remove(to_rmv_id)

        # 2- Add the new objects
        for obj in objPerSubset[to_insert_id]:
            subsetPerObj[obj].append(to_insert_id)
        solution[swapped[0]]=to_insert_id

        #CHECKING VALIDITY
        #obj_invalids=[]
        #tot_score=0
        #for to_check in solution:
        #    for obj in objPerSubset[to_check]:
        #        if not obj in obj_invalids:
        #            obj_invalids.append(obj)
        #            tot_score+=objScores[obj]
        #if tot_score==to_return_real_score:
        #    print("VALID SWAP")
        #else:
        #    print("INVALID SWAPT",tot_score,to_return_real_score)

        # update the tabu list 
        if tabu_list.full():
            tabu_list.get()
            tabu_list.put(to_rmv_id)
        else:
            tabu_list.put(to_rmv_id)
        # and finally end
        return int(to_return_score),to_return_real_score,subsetPerObj,solution,to_return_cost,found
    else:
        return score,score,subsetPerObj,solution,restorer_budget,found
        #if zone==infeasable_zone:
        #    tabu_ctr=0
        #    left_ctr=0
        #    tl=list(tabu_list.queue)
        #    for k in range(len(subsetWeights)):
        #        if k not in solution:
        #            left_ctr+=1
        #            if k in list(tabu_list.queue):
        #                tabu_ctr+=1
        #    print(" OF ", left_ctr, " IN TABU ",tabu_ctr)
            
        
                        

In [7]:
def infeas2feas(curr_score,curr_solution,curr_subsetPerObj,objPerSubset,subsetWeights,objScores,budget):
    # calculate the score(sum of obj/weight) of each element in the solution
    utility_array=[]
    current_weig,to_add_weig,sub_id=0,0,0
    for sub in range(len(curr_solution)):
        sub_id=curr_solution[sub]
        to_add_weig=subsetWeights[sub_id]
        current_weig+=to_add_weig
        #utility_array.append(sum(objScores[obj] for obj in objPerSubset[sub_id])/to_add_weig)
        utility_array.append(sum(objScores[obj] for obj in objPerSubset[sub_id]))
    #print("Making solution feasable score",curr_score, " weig",current_weig)
    while current_weig> budget:
        
        # find the most useless
        useless_idx=np.argmin(utility_array)
        to_rmv=curr_solution[useless_idx]
        # remove it from the utility array
        utility_array.pop(useless_idx)
        # update the score removing obj
        for obj in objPerSubset[to_rmv]:
            #if to_rmv in curr_subsetPerObj[obj]:
            #    curr_subsetPerObj[obj].remove(to_rmv)
            #else:
            #    print("NOT IN LIST,he is ",to_rmv)
            #    print(curr_subsetPerObj[obj])
            curr_subsetPerObj[obj].remove(to_rmv)
            if len(curr_subsetPerObj[obj])==0:
                curr_score-=objScores[obj]
        current_weig-=subsetWeights[to_rmv]
        curr_solution=np.delete(curr_solution,useless_idx)
    #print("MADE solution feasable score",curr_score, " weig",current_weig)
    return (curr_score,curr_subsetPerObj,curr_solution,current_weig)

In [8]:
# Problem data : objPerSubset,subsetWeights,objScores,budget
# Problem solution: score,subsetPerObj,solution
def TabuSearch(objPerSubset,subsetWeights,objScores,budget,score,subsetPerObj,solution,Nt,Ni,tabu_list_size):
    iters_without_updating=0
    iters_infeasable=0
    # current solution values used during tabu search
    current_budget=sum(subsetWeights[sub] for sub in solution)
    curr_score=score
    curr_real_score=curr_score
    curr_solution=solution.copy()
    curr_subsetPerObj=cpy.deepcopy(subsetPerObj)
    # Best values found in the search
    best_budget=current_budget
    best_score=score
    best_solution=curr_solution.copy()
    best_subsetPerObj=cpy.deepcopy(curr_subsetPerObj)
    tabu_list=queue.Queue(tabu_list_size)
    #new_score,new_subset_per_obj,new_solution,new_budget=0,{},[],0
    while iters_without_updating<Nt:
        if iters_infeasable<Ni:
            curr_score,curr_real_score,curr_subsetPerObj,curr_solution,current_budget,found=localSearch(curr_real_score,
                                                                                        curr_solution,curr_subsetPerObj,
                                                                                        objPerSubset,subsetWeights,objScores,
                                                                                         budget,infeasable_zone,tabu_list)
            if not found:
                # TABU DBG
                #print("SHOULD NEVER HAPPEN"," INF ITERS ", iters_infeasable," NO UP ITERS ",iters_without_updating)
                break
        else: 
            # TABU DBG
            #print("Searching in feasable"," INF ITERS ", iters_infeasable," NO UP ITERS ",iters_without_updating)
            curr_score,curr_real_score,curr_subsetPerObj,curr_solution,current_budget,found=localSearch(curr_real_score,
                                                                            curr_solution,curr_subsetPerObj,
                                                                            objPerSubset,subsetWeights,objScores,
                                                                             budget,feasable_zone,tabu_list)
            if not found:
                #TABU DBG
                #print("Making the solution feasable"," INF ITERS ", iters_infeasable," NO UP ITERS ",iters_without_updating,"ACTURAL SCORE",curr_real_score)
                #print(curr_solution)
                (curr_real_score,curr_subsetPerObj,curr_solution,current_budget)=infeas2feas(curr_real_score,curr_solution,
                                                                                           curr_subsetPerObj,objPerSubset,
                                                                                           subsetWeights,objScores,budget)
                # TABU DBG
                #print("New solution",curr_solution,"New score",curr_real_score)
                
                #while not tabu_list.empty():
                #    tabu_list.get()
                
        # in case it is infeasable update the budget
        if current_budget> budget:
            iters_infeasable+=1
            iters_without_updating+=1
        else:
            iters_infeasable=0
            # in case the solution is feasable and better then update
            if curr_real_score> best_score:
                #print("UPDATE")
                iters_without_upating=0
                best_budget=current_budget
                best_score=curr_real_score
                best_solution=curr_solution.copy()
                best_subsetPerObj=cpy.deepcopy(curr_subsetPerObj)
                   
            else:
                iters_without_updating+=1
                #print(" Num of iters ",iters_without_updating)
            
    #print("Returning",best_score)
    return (best_score,best_subsetPerObj,best_solution)

In [9]:
def solutionChecker(score,subsetPerObj,solution,objPerSubset, objScores,budget ):
    weigh_tst,subset_tst,score_tst=False,True,False
    tot_weig=sum(subsetWeights[s] for s in solution)
    if tot_weig<=budget:
        weigh_tst=True
    else:
        weigh_tst=False

    for to_check in solution:
        for obj in objPerSubset[to_check]:
            if not to_check in subsetPerObj[obj]:
                subset_tst=False

    obj_invalids=[]
    tot_score=0
    for to_check in solution:
        for obj in objPerSubset[to_check]:
            if not obj in obj_invalids:
                obj_invalids.append(obj)
                tot_score+=objScores[obj]
    if tot_score==score:
        score_tst=True
    return (weigh_tst,subset_tst,score_tst)

In [10]:
if greedy_tst:
    (objPerSubset,subsetWeights,objScores,budget)=read_txt2( "./instances2/bmcp_1100_1000_0.3_3000.txt")
    (greedy_score,greedy_subsetPerObj,greedy_solution)=greedy(objPerSubset,subsetWeights,objScores,budget)
    print(" The greedy score", greedy_score)
    
    (weigh_tst,subset_tst,score_tst)=solutionChecker(greedy_score,greedy_subsetPerObj,greedy_solution,objPerSubset,objScores,budget)
    if weigh_tst:
        print("Weigh test passed")
    if subset_tst:
        print("Subset test passed")
    if score_tst:
        print("score test passed")

In [11]:
if new_inst_tst:
    (objPerSubset,subsetWeights,objScores,budget)=read_txt2( "./instances2/bmcp_1100_1000_0.3_3000.txt")
    (greedy_score,greedy_subsetPerObj,greedy_solution)=greedy(objPerSubset,subsetWeights,objScores,budget)
    (weigh_tst,subset_tst,score_tst)=solutionChecker(greedy_score,greedy_subsetPerObj,greedy_solution,objPerSubset,objScores,budget)
    print(" The greedy score", greedy_score)
    if weigh_tst:
        print("GREEDY Weigh test passed")
    if subset_tst:
        print(" GREEDY Subset test passed")
    if score_tst:
        print("GREEDY Subset test passed")
    (tabu_score,tabu_subsetPerObj,tabu_solution)=TabuSearch(objPerSubset,subsetWeights,objScores,budget,greedy_score,greedy_subsetPerObj,greedy_solution,200,20,10)

    print("Tabu score",tabu_score)
    (weigh_tst,subset_tst,score_tst)=solutionChecker(tabu_score,tabu_subsetPerObj,tabu_solution,objPerSubset,objScores,budget)
    if weigh_tst:
        print("TABU Weigh test passed")
    if subset_tst:
        print("TABU Subset test passed")
    if score_tst:
        print("TAQU score test passed")
    

 The greedy score 136702
GREEDY Weigh test passed
 GREEDY Subset test passed
GREEDY Subset test passed
Tabu score 138585
TABU Weigh test passed
TABU Subset test passed
TAQU score test passed


In [13]:
if new_inst_tst:
    (objPerSubset,subsetWeights,objScores,budget)=read_txt2( "./instances2/bmcp_1100_1000_0.5_2000.txt")
    (greedy_score,greedy_subsetPerObj,greedy_solution)=greedy(objPerSubset,subsetWeights,objScores,budget)
    (weigh_tst,subset_tst,score_tst)=solutionChecker(greedy_score,greedy_subsetPerObj,greedy_solution,objPerSubset,objScores,budget)
    print(" The greedy score", greedy_score)
    if weigh_tst:
        print("GREEDY Weigh test passed")
    if subset_tst:
        print(" GREEDY Subset test passed")
    if score_tst:
        print("GREEDY Subset test passed")
    (tabu_score,tabu_subsetPerObj,tabu_solution)=TabuSearch(objPerSubset,subsetWeights,objScores,budget,greedy_score,greedy_subsetPerObj,greedy_solution,100,10,5)

    print("Tabu score",tabu_score)
    (weigh_tst,subset_tst,score_tst)=solutionChecker(tabu_score,tabu_subsetPerObj,tabu_solution,objPerSubset,objScores,budget)
    if weigh_tst:
        print("TABU Weigh test passed")
    if subset_tst:
        print("TABU Subset test passed")
    if score_tst:
        print("TAQU score test passed")

 The greedy score 120386
GREEDY Weigh test passed
 GREEDY Subset test passed
GREEDY Subset test passed
Tabu score 122766
TABU Weigh test passed
TABU Subset test passed
TAQU score test passed


In [101]:
if tst_local_search:
    # test of local search
    (objPerSubset,subsetWeights,objScores,budget)=read_txt2(  "./instances2/bmcp_1100_1000_0.5_2000.txt")
    (greedy_score,greedy_subsetPerObj,greedy_solution)=greedy(objPerSubset,subsetWeights,objScores,budget)
    t_l=queue.Queue(10)
    (weigh_tst,subset_tst,score_tst)=solutionChecker(greedy_score,greedy_subsetPerObj,greedy_solution,objPerSubset,objScores,budget)
    if weigh_tst and subset_tst and score_tst:
        print("GREEDY VALID")
    curr_score,curr_real_score,curr_subsetPerObj,curr_solution,current_budget,found=localSearch(greedy_score,
                                                                                        greedy_solution,greedy_subsetPerObj,
                                                                                        objPerSubset,subsetWeights,objScores,
                                                                                         budget,infeasable_zone,t_l)
    (weigh_tst,subset_tst,score_tst)=solutionChecker(curr_real_score,curr_subsetPerObj,curr_solution,objPerSubset,objScores,budget)
    print(weigh_tst,subset_tst,score_tst)
    print(" F2 ", curr_score, " F1 ", curr_real_score)

In [102]:
if tst_local_search:
    # test of local search
    (objPerSubset,subsetWeights,objScores,budget)=read_txt2(  "./instances2/bmcp_1100_1000_0.3_3000.txt")
    (greedy_score,greedy_subsetPerObj,greedy_solution)=greedy(objPerSubset,subsetWeights,objScores,budget)
    t_l=queue.Queue(10)
    (weigh_tst,subset_tst,score_tst)=solutionChecker(greedy_score,greedy_subsetPerObj,greedy_solution,objPerSubset,objScores,budget)
    if weigh_tst and subset_tst and score_tst:
        print("GREEDY VALID")
    curr_score,curr_real_score,curr_subsetPerObj,curr_solution,current_budget,found=localSearch(greedy_score,
                                                                                        greedy_solution,greedy_subsetPerObj,
                                                                                        objPerSubset,subsetWeights,objScores,
                                                                                         budget,infeasable_zone,t_l)
    (weigh_tst,subset_tst,score_tst)=solutionChecker(curr_real_score,curr_subsetPerObj,curr_solution,objPerSubset,objScores,budget)
    print(weigh_tst,subset_tst,score_tst)
    print(" F2 ", curr_score, " F1 ", curr_real_score)